In [76]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

import warnings
import functools
import wrds
import sys
# Ignore all warnings
warnings.filterwarnings("ignore")

In [77]:
df_analysis = pd.read_pickle('results_shrey.pkl')
# df_analysis

In [78]:
sectors_filter = pd.read_excel('tickers_by_sector.xlsx', sheet_name='earnings_dates_for_tickers')
sectors = pd.read_excel('tickers_by_sector.xlsx', sheet_name='Sheet1')

In [79]:
sectors = sectors.drop(sectors.index[0])
sectors.columns = sectors.iloc[0]
sectors = sectors.drop(sectors.index[0])

In [80]:
sectors = sectors.rename(columns={'short ticker': 'ticker'})
sectors.columns

Index(['Ticker', 'Short Name', 'Market Cap', 'Price:D-1', 'P/E',
       'Total Return YTD', 'Revenue T12M', 'EPS T12M', 'ticker new format',
       'ticker', 'SECTOR'],
      dtype='object', name=1)

In [81]:
sectors_tickers = sectors[['ticker', 'SECTOR']]
# sectors_tickers

In [82]:
import re
def remove_text_within_parentheses_and_numbers(text):
    # Verifica si el valor es NaN (float) y retorna NaN si es el caso
    if pd.isnull(text):
        return text
    # Elimina cualquier texto entre paréntesis incluyendo los paréntesis mismos y números
    return re.sub(r"\s*\([^)]*\)|\d+", "", text)

# Aplica la función a la columna 'SECTOR'
sectors_tickers['SECTOR'] = sectors_tickers['SECTOR'].apply(remove_text_within_parentheses_and_numbers)
sectors_tickers.rename(columns={'ticker':'Short_ticker'}, inplace=True)



In [83]:
df_analysis = pd.merge(df_analysis, sectors_tickers, on='Short_ticker', how='left') 
# df_analysis

In [84]:
def count_peaks(data, buffer_pct = 20):
    peaks = 0
    for i in range(1, len(data) - 1):
        if (1 + buffer_pct/100)*data[i-1] < data[i] > (1 + buffer_pct/100)*data[i+1]:
            peaks += 1
    return peaks

for buffer_p in [10,15,20,25]:
    df_analysis[f'num_peaks_c_w_buffer_{buffer_p}'] = df_analysis['IV_c'].apply(lambda x: count_peaks(x.dropna(how='any', axis=0)['impl_volatility'],buffer_p))
    df_analysis[f'num_peaks_p_w_buffer_{buffer_p}'] = df_analysis['IV_p'].apply(lambda x: count_peaks(x.dropna(how='any', axis=0)['impl_volatility'],buffer_p))
    
bins = [0, 1, 2, float('inf')]
labels = ['0', '1', '>=2']

for buffer_p in [10,15,20,25]:
    df_analysis[f'num_peaks_call_bucket_{buffer_p}'] = pd.cut(df_analysis[f'num_peaks_c_w_buffer_{buffer_p}'], bins=bins, labels=labels, right=False, include_lowest=True)
    df_analysis[f'num_peaks_put_bucket_{buffer_p}'] = pd.cut(df_analysis[f'num_peaks_p_w_buffer_{buffer_p}'], bins=bins, labels=labels, right=False, include_lowest=True)



In [85]:
df_analysis['num_put_strikes'].describe()

count    3614.000000
mean       30.074433
std        19.292101
min         6.000000
25%        18.000000
50%        25.000000
75%        36.000000
max       250.000000
Name: num_put_strikes, dtype: float64

In [86]:
num_peaks_call_desc = df_analysis.groupby('SECTOR')['num_peaks_call_bucket_10'].value_counts(normalize=True)
num_peaks_put_desc = df_analysis.groupby('SECTOR')['num_peaks_put_bucket_10'].value_counts(normalize=True)
display(pd.DataFrame(num_peaks_call_desc))
display(pd.DataFrame(num_peaks_put_desc))

proportion
SECTOR                 num_peaks_call_bucket_10            
Communication Services 0                           0.551220
                       1                           0.312195
                       >=2                         0.136585
Consumer Discretionary 0                           0.637485
                       1                           0.229346
                       >=2                         0.133169
Consumer Staples       0                           0.678977
                       1                           0.218750
                       >=2                         0.102273
Energy                 0                           0.739583
                       1                           0.171875
                       >=2                         0.088542
Financials             0                           0.611285
                       1                           0.282132
                       >=2                         0.106583
Health Care            0                           0.547945
                       1                           0.285388
                       >=2                         0.166667
Industrials            0                           0.684564
                       1                           0.217002
                       >=2                         0.098434
Information Technology 0                           0.514143
                       1                           0.292845
                       >=2                         0.193012
Materials              0                           0.751553
                       1                           0.186335
                       >=2                         0.062112
PACS US Equity         0                           0.400000
                       >=2                         0.350000
                       1                           0.250000
Real Estate            0                           0.857143
                       1                           0.142857
                       >=2                         0.000000
Utilities              0                           0.888889
                       1                           0.111111
                       >=2                         0.000000

proportion
SECTOR                 num_peaks_put_bucket_10            
Communication Services 0                          0.512195
                       1                          0.356098
                       >=2                        0.131707
Consumer Discretionary 0                          0.628853
                       1                          0.260173
                       >=2                        0.110974
Consumer Staples       0                          0.653409
                       1                          0.255682
                       >=2                        0.090909
Energy                 0                          0.723958
                       1                          0.171875
                       >=2                        0.104167
Financials             0                          0.645768
                       1                          0.238245
                       >=2                        0.115987
Health Care            0                          0.470320
                       1                          0.353881
                       >=2                        0.175799
Industrials            0                          0.606264
                       1                          0.277405
                       >=2                        0.116331
Information Technology 0                          0.579035
                       1                          0.274542
                       >=2                        0.146423
Materials              0                          0.677019
                       1                          0.211180
                       >=2                        0.111801
PACS US Equity         0                          0.500000
                       1                          0.300000
                       >=2                        0.200000
Real Estate            0                          0.571429
                       1                          0.428571
                       >=2                        0.000000
Utilities              0                          0.777778
                       1                          0.222222
                       >=2                        0.000000

In [87]:
num_peaks_call_desc = df_analysis.groupby('SECTOR')['num_peaks_call_bucket_20'].value_counts(normalize=True)
num_peaks_put_desc = df_analysis.groupby('SECTOR')['num_peaks_put_bucket_20'].value_counts(normalize=True)
display(pd.DataFrame(num_peaks_call_desc))
display(pd.DataFrame(num_peaks_put_desc))

proportion
SECTOR                 num_peaks_call_bucket_20            
Communication Services 0                           0.800000
                       1                           0.146341
                       >=2                         0.053659
Consumer Discretionary 0                           0.808878
                       1                           0.151665
                       >=2                         0.039457
Consumer Staples       0                           0.809659
                       1                           0.153409
                       >=2                         0.036932
Energy                 0                           0.854167
                       1                           0.114583
                       >=2                         0.031250
Financials             0                           0.796238
                       1                           0.169279
                       >=2                         0.034483
Health Care            0                           0.746575
                       1                           0.189498
                       >=2                         0.063927
Industrials            0                           0.836689
                       1                           0.134228
                       >=2                         0.029083
Information Technology 0                           0.740433
                       1                           0.183028
                       >=2                         0.076539
Materials              0                           0.863354
                       1                           0.105590
                       >=2                         0.031056
PACS US Equity         0                           0.650000
                       1                           0.250000
                       >=2                         0.100000
Real Estate            0                           1.000000
                       1                           0.000000
                       >=2                         0.000000
Utilities              0                           0.888889
                       1                           0.111111
                       >=2                         0.000000

proportion
SECTOR                 num_peaks_put_bucket_20            
Communication Services 0                          0.770732
                       1                          0.185366
                       >=2                        0.043902
Consumer Discretionary 0                          0.823674
                       1                          0.143033
                       >=2                        0.033292
Consumer Staples       0                          0.835227
                       1                          0.133523
                       >=2                        0.031250
Energy                 0                          0.854167
                       1                          0.125000
                       >=2                        0.020833
Financials             0                          0.818182
                       1                          0.147335
                       >=2                        0.034483
Health Care            0                          0.714612
                       1                          0.232877
                       >=2                        0.052511
Industrials            0                          0.821029
                       1                          0.134228
                       >=2                        0.044743
Information Technology 0                          0.795341
                       1                          0.171381
                       >=2                        0.033278
Materials              0                          0.819876
                       1                          0.149068
                       >=2                        0.031056
PACS US Equity         0                          0.650000
                       1                          0.200000
                       >=2                        0.150000
Real Estate            0                          1.000000
                       1                          0.000000
                       >=2                        0.000000
Utilities              0                          0.888889
                       1                          0.111111
                       >=2                        0.000000

In [88]:
df_ge_2_c = pd.DataFrame()
for buffer_p in [10,15,20,25]:
    num_peaks_call_pnl = df_analysis.groupby(f'num_peaks_call_bucket_{buffer_p}')['straddle_pnl'].describe()
    # print(buffer_p)
    # display(num_peaks_call_pnl.T)
    df_ge_2_c = pd.concat([df_ge_2_c, num_peaks_call_pnl.T['>=2']], axis=1)
    # print('\n')

df_ge_2_c.columns = ['>=2_10', '>=2_15', '>=2_20', '>=2_25']
df_ge_2_c.loc['count'] = df_ge_2_c.loc['count'] / df_analysis.shape[0]
df_ge_2_c

,>=2_10,>=2_15,>=2_20,>=2_25
count,0.133924,0.069729,0.046763,0.027117
mean,13.962624,28.451615,46.574050,128.313720
std,640.209994,651.688926,558.195409,496.452083
min,-4520.470588,-4520.470588,-2267.379679,-1573.347324
25%,-263.730766,-240.000291,-240.458015,-211.930105
50%,201.626016,226.760543,209.876543,287.375477
75%,433.576075,454.836003,445.283019,508.774038
max,918.224299,918.224299,918.224299,886.446886


In [89]:
df_ge_2_p = pd.DataFrame()
for buffer_p in [10,15,20,25]:
    num_peaks_put_pnl = df_analysis.groupby(f'num_peaks_put_bucket_{buffer_p}')['straddle_pnl'].describe()
    # print(buffer_p)
    # display(num_peaks_call_pnl.T)
    df_ge_2_p = pd.concat([df_ge_2_p, num_peaks_put_pnl.T['>=2']], axis=1)
    # print('\n')

df_ge_2_p.columns = ['>=2_10', '>=2_15', '>=2_20', '>=2_25']
df_ge_2_p.loc['count'] = df_ge_2_p.loc['count'] / df_analysis.shape[0]
df_ge_2_p

,>=2_10,>=2_15,>=2_20,>=2_25
count,0.126453,0.064471,0.037908,0.023796
mean,25.222713,23.370469,56.305233,62.125591
std,601.641276,567.891760,515.699656,534.767185
min,-3352.654867,-2267.379679,-1726.550079,-1658.536585
25%,-243.902439,-214.446953,-201.117318,-90.641711
50%,198.369565,214.149140,226.872247,225.760067
75%,437.272727,404.392765,401.360544,394.066515
max,918.224299,918.224299,749.015748,749.015748
